# Data exploration

In [1]:
import laspy
from scipy.spatial import cKDTree
import numpy as np
from sklearn.decomposition import PCA

In [2]:
def get_spatial_subset(
    full_cloud, xmin: int, xmax: int, ymin: int, ymax: int
) -> laspy.lasdata.LasData:
    """
    Create a subset of a LAS file based on a bounding box in coordinates of the cloud file

    parameters:
    full_cloud: a laspy cloud object you want to subset from
    xmin,xmax,ymin

    returns: laspy.lasdata.LasData object
    """
    # create empty laspy collection to put the filtered points in that has the same format and file version as the original
    new_file = laspy.create(
        point_format=cloud.header.point_format, file_version=cloud.header.version
    )
    # create matrices of boolean values
    a = cloud.x > xmin
    b = cloud.x < xmax
    c = cloud.y > ymin
    d = cloud.y < ymax
    # subset the points and put them into the new laspy file
    new_file.points = full_cloud.points[a & b & c & d]
    return new_file


In [3]:
# load in the fully LAZ file
cloud = laspy.read(r'./data/AHN4_Noordwijk.laz')


In [4]:
# define the area of interest
xcenter = 89400
bbbox_size = 400
ycenter = 472868
xmin = xcenter - 0.5*bbbox_size
xmax = xcenter + 0.5*bbbox_size
ymin = ycenter - 0.5*bbbox_size
ymax = ycenter + 0.5*bbbox_size
print(f'Bounding box is between {xmin} and {xmax} in the x direction and {ymin} and {ymax} in the y direction (referenced to the coordinates in the LAZ file)')

Bounding box is between 89200.0 and 89600.0 in the x direction and 472668.0 and 473068.0 in the y direction (referenced to the coordinates in the LAZ file)


In [5]:
subset = get_spatial_subset(cloud,xmin,xmax,ymin,ymax)
subset.write('.\data\subsetted.las')

In [6]:
dataset = np.vstack((subset.X,subset.Y,subset.Z)).transpose()
tree = cKDTree(dataset)

In [7]:
# going to test this algorithm on a smaller subset Of the subset so it runs faster
test_set = subset.points.array[10:2000]

In [8]:
# setting up the extra dimensions to add back to the las file
dimension = laspy.point.format.ExtraBytesParams('normal','float64')
subset.add_extra_dim(dimension)
list(subset.point_format.extra_dimension_names)

['Deviation', 'Reflectance', 'Amplitude', 'normal']

In [9]:
# set up an empty array
normals = np.zeros(len(test_set))
for i,point in enumerate(test_set):
    x = point[0]
    y = point[1]
    z = point[2]
    neighbors_distance, neighbors_indices = tree.query((x,y,z),k=8)
    neighbors_points = tree.data[neighbors_indices]
    # set up pca model
    pca = PCA(n_components=3)

    pca.fit(neighbors_points)
    # get the Z value for the normal
    normal_vector = pca.components_.T[2]

    angle = np.rad2deg(np.arccos(normal_vector.dot([0,0,1])))
    normals[i] = angle
    

a) Investigate the data set: describe each of its columns and assess the spread in values in each column. What is the meaning of each attribute?

| Column name | max | min | average | description |
| ----------- | --- | --- | ------- | ----------- |
|  |  |  |  |  |





b) What are ways to create a subset? Select a suitable subset of the data and visualize it in 3D. Start small, and if computer and software permits, try a bit larger subset


c) Identify at least four different object classes. Choose your classes such that together these cover a large majority of your points.

1. building
2. grass
3. pavement
4. tree
5. beach

d) Analyse which spatial scales and spectral properties are useful to distinguish your classes.

The following properties are typical of each of the classes

1. building
   1. taller than the surroundings 
2. grass
   1. high NDVI
   2. 
3. pavement
   1. 
4. tree
   1. taller that it is wide or long
5. beach

e) Extract training data for each of your object classes. Divide your training data in two parts, one for training, and one for validation. What could be the influence of  imbalances in your training data? How could your division in training and validation data affect your results?


f) Find a suitable implementation of the Random Forest algorithm. What are its parameters? What would be good settings for these parameters, given your classification task? Apply Random Forest on your data using only your observed features, to make sure your setup is correct.



# Questions
1. Describe the properties of the data (sub)set that you will classify: (i) How many points? (ii) What area does it cover? (iii) What observed features will you use? (iv) Visualize your final subset, including the useful observed features. (vi) Describe your at least four different object classes; For each point determine neighborhoods of k1, k2, ... points (using e.g. an efficient data structure), and use these k1, k2, ... points to estimate several feature values.


2. Describe $\gt$ 20 different geometric attributes, obtained using at least 2 different neighborhood sizes, to characterize your points. What are the dimensions of the data covariance matrix you use to compute the geometric features? Give one example on how you determine the PCA eigenvalues of one k-neighborhood. Indicate for each feature how it could help to distinguish your classes, given also the neighborhood sizes you consider.
   

3. Compute all geometric features for all points in your subset using Python. Visualize selected results, e.g. by combining features in a false color visualization and/or using histograms. Which features are best at discriminating your classes? Why?

4. Describe and visualize your training data. Is your training data balanced? Make sure that a zone of your point cloud data is really ’unseen’, that is that no training data is taken from that zone, so you can inspect if Random Forest also works there.
   

5. Feed your training data to Random Forest using at least 10 of your best geometric and observed features. What settings did you use?
   

6. Classify your point cloud data and visualize and discuss your result. What went well? Give also examples where the classifier mixed up classes. What are possible explanations for these confusions? How are the classification results on the unseen zone?